In [5]:
from eventvision import read_dataset, read_annotation

import os
import cv2
import glob
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
from scipy.stats import gaussian_kde
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from BoundingBoxRegressionUtils import makeModel, parse_partial_dataset, parse_total_dataset, parse_random_dataset, tf_iou

POINTS = 500
INPUT_SHAPE = keras.Input(shape=(POINTS, 3, 1))

In [2]:
def makeBB(filename, t):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    return [minX, minX, maxX, maxX, minX], [minY, maxY, maxY, minY, minY], [t,t,t,t,t]

def getData(filename):
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    
    points = np.column_stack((td.data.x[:POINTS], td.data.y[:POINTS], td.data.ts[:POINTS] / 1000))
    
    return width, height, np.array(points).reshape(POINTS, 3, 1)

def getAnnotation(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    return minX, minY, maxX, maxY


In [3]:
def testDataTime():
    data_results = np.empty(20)
    
    for i in range(20):
        data_start_time = time.process_time() 

        X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        data_time = time.process_time() - data_start_time
        
        data_results[i] = data_time
    
    return np.mean(data_results)

# print("data avg", testDataTime())

In [6]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def test(model, test_data):
    predictions = model.predict(test_data[0])
    
    bb_scores = tf_iou(predictions[:, -4:], test_data[1][:, -4:])
        
    return np.mean(bb_scores)

def makeTestData():
    testData = []
    
    for i in range(10):
        testData.append(parse_random_dataset(getData, getAnnotation, True))
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_data = testData[i]

        testing_time = time.time()

        iou = test(model, test_data)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = makeModel(INPUT_SHAPE, 8)
    
    train_start_time = time.process_time()

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

Round  0
Training done!  16.359375
	For this mini round, time:  0.17599868774414062  iou  0.6861577
	For this mini round, time:  0.11902976036071777  iou  0.6887052
	For this mini round, time:  0.1139683723449707  iou  0.7001675
	For this mini round, time:  0.11802840232849121  iou  0.68718535
	For this mini round, time:  0.11299991607666016  iou  0.6978564
	For this mini round, time:  0.11600232124328613  iou  0.70259345
	For this mini round, time:  0.11799335479736328  iou  0.69968957
	For this mini round, time:  0.12697505950927734  iou  0.69426584
	For this mini round, time:  0.11799979209899902  iou  0.696821
	For this mini round, time:  0.11403131484985352  iou  0.69401366
Time on avg:  0.12330269813537598  IOU on avg:  0.6947455644607544
Round  1
Training done!  16.46875
	For this mini round, time:  0.170029878616333  iou  0.67409956
	For this mini round, time:  0.11596941947937012  iou  0.6770782
	For this mini round, time:  0.12100076675415039  iou  0.68708885
	For this mini r

Training done!  15.640625
	For this mini round, time:  0.17799830436706543  iou  0.68997186
	For this mini round, time:  0.12600111961364746  iou  0.6913987
	For this mini round, time:  0.12502646446228027  iou  0.70076007
	For this mini round, time:  0.12099933624267578  iou  0.69010556
	For this mini round, time:  0.12003469467163086  iou  0.6994204
	For this mini round, time:  0.12400102615356445  iou  0.7029618
	For this mini round, time:  0.12699413299560547  iou  0.70143056
	For this mini round, time:  0.12799715995788574  iou  0.698179
	For this mini round, time:  0.12897276878356934  iou  0.6991808
	For this mini round, time:  0.13399004936218262  iou  0.6923135
Time on avg:  0.13120150566101074  IOU on avg:  0.69657222032547
Round  12
Training done!  16.46875
	For this mini round, time:  0.1679983139038086  iou  0.6787213
	For this mini round, time:  0.11899447441101074  iou  0.6814436
	For this mini round, time:  0.12497687339782715  iou  0.6892828
	For this mini round, time:

(array([16.359375, 16.46875 , 15.921875, 15.75    , 16.0625  , 15.78125 ,
        16.5625  , 16.375   , 16.265625, 16.734375, 16.109375, 15.640625,
        16.46875 , 16.171875, 16.640625, 16.328125, 17.21875 , 16.296875,
        15.96875 , 15.65625 ]),
 array([0.1233027 , 0.12639806, 0.1233001 , 0.1253108 , 0.12520809,
        0.1309994 , 0.14479973, 0.12460008, 0.13110273, 0.12670135,
        0.12620299, 0.13120151, 0.12409399, 0.15280952, 0.12549961,
        0.13570302, 0.12839749, 0.1254998 , 0.12759974, 0.12350032]),
 array([0.69474556, 0.68132728, 0.70534365, 0.67451701, 0.69236153,
        0.65197117, 0.7093196 , 0.67595845, 0.69999831, 0.68000879,
        0.70265552, 0.69657222, 0.68676879, 0.69595001, 0.68065242,
        0.70198353, 0.69002681, 0.68666832, 0.6894662 , 0.68396383]))

In [ ]:
def trainModel():
    model = makeModel(INPUT_SHAPE, 8)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)
    
    return history, model

history, model = trainModel()

In [ ]:
w, h, points = getData(os.getcwd() + r'\..\Caltech101\airplanes\image_0002.bin')

inputPoints = points.reshape([1, POINTS, 3, 1])
preds = model.predict(inputPoints)[0]
className = np.argmax(preds[:4])

minX = preds[4] * w
minY = preds[5] * h
maxX = preds[6] * w
maxY = preds[7] * h


fig = plt.figure(figsize=(20,15))
ax = plt.axes(projection ="3d")

ax.scatter3D(points[:,2], points[:,0], points[:,1])

for i in range(0, 15, 2):
    x,y,z = makeBB(os.getcwd() + r'\..\Caltech101_annotations\airplanes\annotation_0002.bin', i)
    ax.plot3D(z,x,y, "red")
    ax.plot3D([i,i,i,i,i],[minX, minX, maxX, maxX, minX],[minY, maxY, maxY, minY, minY], "green")

# plt.imshow(points)
# ax = plt.gca()
# ax.add_patch(makeBB(r'C:\Users\Pasca\N-MNIST\Caltech101_annotations\airplanes\annotation_0002.bin'))
# ax.add_patch(Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='green', facecolor='none', label="predicted"))
# ax.text(2000,maxX, (maxY), str(className),verticalalignment='top',color='black',fontsize=20,weight='bold')
# ax.legend(loc="upper left", prop={'size': 15})

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

print(np.mean(history.history['val_loss']))
print(np.mean(history.history['val_accuracy']))